In questo progetto in preparazione all'esame di Deep learning ho pensato di affrontare un problema di classificazione.
In questo progetto, sfruttando un dataset disponibile online riguardante i frutti, esploreremo i dati a disposizione, addestreremo un sistema alla classificazione dei frutti e etesteremo altre funzionalità come la creazione di immagini di frutti.

Montiamo Google Drive per usarlo con il notebook per contenere codice e dati.

In [ ]:
# SOLO SU GOOGLE COLAB
from google.colab import drive
drive.mount('/content/drive')

# **Ottenere il dataset per l'addestramento**
Scarichiamo il dataset per l'addestramento sfruttando da kaggle e lo decomprimiamo sfruttando la possibilità offerta da Google Colab di inviare istruzioni shell bash

In [ ]:
# SOLO SU GOOGLE COLAB
!mkdir -p /content/drive/MyDrive/FruitDLProj # Crea il path qualora non esista

In [ ]:
# SOLO SU GOOGLE COLAB
!curl -L -o /content/drive/MyDrive/FruitDLProj/fruits.zip\
  https://www.kaggle.com/api/v1/datasets/download/moltean/fruits

In [ ]:
# SOLO SU GOOGLE COLAB
!unzip /content/drive/MyDrive/FruitDLProj/fruits.zip -d /content/drive/MyDrive/FruitDLProj/frutti

In [ ]:
# SOLO SU GOOGLE COLAB
!ls /content/drive/MyDrive/FruitDLProj/frutti

# **Esplorazione dei dati**
Iniziamo l'esplorazione del dataset:
*   valuteremo la sua divisione in train e Test ricavando il numero di classi e le etichette associate dalla struttura dei file.
*   Valuteremo la suddivisione delle entry per ogni classe per Train e test
*   Rappresenteremo con un grafico a barre questa suddivisione
*   Vedremo un raffronto Train/Test per le 5 classi più popolate
*   Costruiremo una nuvola di punti di un campione di dati per apprezzare visivamente la distribuzione delle entry in gruppi distinti, nel fare questo dovremo ridurre la dimensionalità dei dati al fine della visualizzazione grafica.







In [ ]:
import os # Libreria per interagire con l'OS
import matplotlib.pyplot as plt # Libreria per la generazione di grafici
import seaborn as sns # Libreria per dei grafici visivamente più efficaci
import pandas as pd # Libreria per la manipolazione e l'analisi dei dati tabellari
import matplotlib.image as mpimg # Modulo specifico di Matplotlib per caricare e manipolare immagini
import numpy as np # Libreria fondamentale per il calcolo numerico con funzioni quali la manipolazione dei tensori
from collections import Counter # Libreria utile per contare gli elementi in una collezione
from PIL import Image # Libreria per caricare e manipolare le immagini
from sklearn.manifold import TSNE # Libreria per la riduzione di dimensionalità
from sklearn.preprocessing import StandardScaler # Libreria per scalare i dati prima di t-SNE
import random # Libreria che fornisce funzionalità per generare numeri casuali e per operare con elementi casuali
import time # Libreria per misurare il tempo di esecuzione di diverse parti del tuo codice utile per monitorare le prestazion

# 1. Definizione dei percorsi del dataset
# path su Google Colab
#dataset_root = '/content/drive/MyDrive/FruitDLProj/frutti/fruits-360_100x100/fruits-360'
dataset_root = '/kaggle/input/fruits/fruits-360_100x100/fruits-360'

# Suddivisione dei percorsi con i dati di train e di test
train_dir = os.path.join(dataset_root, 'Training')
test_dir = os.path.join(dataset_root, 'Test')

# 2. Preparazione dei dati per i grafici
# Otteniamo le classi dai nomi delle cartelle

# Ordiniamo le classi (tipi di frutta)
train_classes = sorted(os.listdir(train_dir))
test_classes = sorted(os.listdir(test_dir))

# Contiamo il numero di classi
num_train_classes = len(train_classes)
num_test_classes = len(test_classes)

print(f"Numero di classi nel Training set: {num_train_classes}")
print(f"Numero di classi nel Test set: {num_test_classes}")
print(f"Le classi nel Training set sono: {train_classes[:5]} ... (mostro solo le prime 5)") # Esempio delle prime 5 classi
print("-" * 30)

# Verifichiamo se le classi sono le stesse nei due set
if train_classes == test_classes:
    print("Le classi nel training e nel test set corrispondono.")
    all_classes = train_classes
else:
    print("Attenzione: Le classi nel training e nel test set NON corrispondono completamente.")
    all_classes = sorted(list(set(train_classes + test_classes))) # Unisce ed ordina le classi uniche in una lista

if not all_classes:
    print(f"Errore: Nessuna classe trovata nella directory di training e test: {train_dir}, {test_dir}")
    print("Controlla il percorso del dataset.")
else:
    print(f"Trovate {len(all_classes)} classi di frutta.")


# Contiamo le immagini per classe nel Training Set
train_image_counts = {}
for fruit_class in all_classes:
    class_path = os.path.join(train_dir, fruit_class)
    if os.path.exists(class_path) and os.path.isdir(class_path):
        train_image_counts[fruit_class] = len(os.listdir(class_path))
    else:
        train_image_counts[fruit_class] = 0 # In caso la classe non esista o non sia una directory

# Contiamo le immagini per classe nel Test Set
test_image_counts = {}
for fruit_class in all_classes:
    class_path = os.path.join(test_dir, fruit_class)
    if os.path.exists(class_path) and os.path.isdir(class_path):
        test_image_counts[fruit_class] = len(os.listdir(class_path))
    else:
        test_image_counts[fruit_class] = 0 # In caso la classe non esista o non sia una directory

# Converto i dizionari in DataFrame per facilitare la creazione dei grafici
df_train = pd.DataFrame(train_image_counts.items(), columns=['Classe', 'Numero Immagini'])
df_test = pd.DataFrame(test_image_counts.items(), columns=['Classe', 'Numero Immagini'])

# Ordino per numero di immagini per una migliore visualizzazione
df_train = df_train.sort_values(by='Numero Immagini', ascending=False)
df_test = df_test.sort_values(by='Numero Immagini', ascending=False)


# **Suddivisione delle entry per classe nei Train Set e Test Set**

In [ ]:
# 3. Creazione dei grafici a barre (Barplot)

# Seleziona solo le prime 15 classi per una miglior visualizzazione
df_train_top = df_train.head(15)
df_test_top = df_test.head(15)

plt.style.use('seaborn-v0_8-darkgrid') # Applichiamo lo stile al grafico usando seaborn

# Grafico per il Training Set
plt.figure(figsize=(25, 9)) # Aumentiamo la dimensione del grafico per leggibilità
sns.barplot(x='Classe', y='Numero Immagini', data=df_train_top, palette='viridis', hue='Classe')
plt.title('Distribuzione Immagini per Classe nel Training Set', fontsize=28) # Titolo del grafico
plt.xlabel('Classe di Frutta', fontsize=22) # Etichetta dell'asse X
plt.ylabel('Numero di Immagini', fontsize=22) # Etichetta dell'asse Y
plt.xticks(rotation=90, fontsize=18) # Ruota le etichette dell'asse X per evitare sovrapposizioni
plt.tight_layout() # Adatta il layout per evitare tagli
plt.show()

# Grafico per il Test Set
plt.figure(figsize=(25, 9))
sns.barplot(x='Classe', y='Numero Immagini', data=df_test_top, palette='plasma', hue='Classe')
plt.title('Distribuzione Immagini per Classe nel Test Set', fontsize=28)
plt.xlabel('Classe di Frutta', fontsize=22)
plt.ylabel('Numero di Immagini', fontsize=22)
plt.xticks(rotation=90, fontsize=18)
plt.tight_layout()
plt.show()

Visto il gran numero di classi selezioniamo solo le 5 più popolate nel training set in modo da avere un livello di dettaglio che ci permetta di osservare il rapporto tra gli elementi nei set di Train e di Test

In [ ]:
# Kaggle notebooks genera numerosi warning a causa dell'uso di versioni precedenti di python
# per cui gli andiamo a sopprimere
import warnings
warnings.filterwarnings('ignore')

# Seleziono le 5 classi più popolose dal training set
top_5_train_classes = df_train.head(5)['Classe'].tolist()

# Filtro entrambi i DataFrame per includere solo le 5 classi selezionate
df_train_filtered = df_train[df_train['Classe'].isin(top_5_train_classes)].copy()
df_test_filtered = df_test[df_test['Classe'].isin(top_5_train_classes)].copy()

# Aggiungi una colonna 'Set' per identificare l'origine dei dati che poi unirò in un unico dataframe
df_train_filtered['Set'] = 'Train'
df_test_filtered['Set'] = 'Test'

# Combino i due DataFrame filtrati in un unico DataFrame
df_combined_top5 = pd.concat([df_train_filtered, df_test_filtered])

# Riordino le classi in base all'ordine del training set per la visualizzazione nel grafico
df_combined_top5['Classe'] = pd.Categorical(df_combined_top5['Classe'], categories=top_5_train_classes, ordered=True)
df_combined_top5 = df_combined_top5.sort_values('Classe')

plt.figure(figsize=(12, 7))
# Uso 'hue' per distinguere tra 'Train' e 'Test'
sns.barplot(x='Classe', y='Numero Immagini', hue='Set', data=df_combined_top5, palette='viridis') # Palette automatica per hue
plt.title('Distribuzione Immagini per le 5 Classi Top del Training Set (Train vs. Test)', fontsize=14)
plt.xlabel('Classe di Frutta', fontsize=12)
plt.ylabel('Numero di Immagini', fontsize=12)
plt.xticks(rotation=45, ha='right', fontsize=10) # Ruota le etichette per leggibilità
plt.grid(axis='y', linestyle='--', alpha=0.7)
plt.legend(title='Dataset') # La leggenda viene generata automaticamente da hue
plt.tight_layout()
plt.show()

# **Riduzione dimensionale e classificazione visiva della nuvola di punti**
Nell'esplorazione ed analisi dei dati vogliamo ora vedere le singole entry del dataset come nuvola di punti per una stima visiva del loro raggruppamento in classi. Per visualizzare i diversi elementi, che nel nostro caso sono delle immagini, come una nuvola di punti (scatter plot) e far emergere il loro raggruppamento in classi, non possiamo semplicemente prendere le immagini così come sono in quanto sono dati ad alta dimensione (ad esempio, un'immagine 100x100 pixel RGB significa 30.000 dimensioni!), e non possiamo plottare un punto in uno spazio così grande.

Dobbiamo prima ridurre la dimensionalità delle immagini a 2 o 3 dimensioni, mantenendo il più possibile le relazioni spaziali tra i dati.

Per fare questo usiamo la tecnica t-SNE ma vediamo prima alcune tecniche comuni:

* Principal Component Analysis (PCA): Lineare, cerca di trovare le direzioni di massima varianza nei dati.
* t-distributed Stochastic Neighbor Embedding (t-SNE): Non lineare, eccellente per visualizzare raggruppamenti di dati ad alta dimensione in uno spazio a bassa dimensione, preservando le vicinanze tra i punti. È spesso preferito per la visualizzazione.
* Uniform Manifold Approximation and Projection (UMAP): Simile a t-SNE, spesso più veloce e scalabile, e talvolta produce raggruppamenti più densi.

Per il nostro scopo, abbiamo scelto t-SNE in quanto ideale per la visualizzazione di raggruppamenti.

In [ ]:
# Su Kaggle notebooks non riesco a sopprimere i warning generati in questa sezione
# ma fino all'addestramento funziona tutto anche su Google Colab sostituendo il path
# del dataset

# 4. Impostazione dei parametri per l'elaborazione della nuvola di punti
IMAGE_SIZE = (64, 64) # Ridimensiona le immagini a una dimensione più gestibile
MAX_IMAGES_PER_CLASS = 100 # Riduciamo il numero massimo di immagini da caricare per classe
NUM_CLASSES_TO_PLOT = 10   # Limitiamo la visualizzazione a sole 10 classi

print(f"Caricamento dati da: {train_dir}")
print(f"Ridimensionamento immagini a: {IMAGE_SIZE[0]}x{IMAGE_SIZE[1]}")
print(f"Max immagini per classe: {MAX_IMAGES_PER_CLASS}\n")
print(f"Numero di classi da mostrare nel grafico: {NUM_CLASSES_TO_PLOT}\n")

# 4.1. Caricamento e pre-processing di un sottoinsieme delle immagini
data = []
labels = []
class_names = sorted(os.listdir(train_dir))
if not class_names:
    print(f"Errore: Nessuna classe trovata nella directory di training: {train_dir}")
    print("Controlla il percorso del dataset e assicurati che sia scompattato.")
    exit()

# Selezioniamo solo le prime N classi per il nostro esempio
class_names_to_process = class_names[:NUM_CLASSES_TO_PLOT]

print(f"Caricamento immagini dalle prime {NUM_CLASSES_TO_PLOT} classi: {class_names_to_process} in corso...")
start_time = time.time()

for i, class_name in enumerate(class_names_to_process):
    class_path = os.path.join(train_dir, class_name)
    if os.path.isdir(class_path):
        images_in_class = [img for img in os.listdir(class_path) if img.endswith(('.jpg', '.jpeg', '.png'))]
        random.shuffle(images_in_class) # Mescola le entry per selezionare un sottoinsieme casuale

        # Limita il numero di immagini per classe
        for img_name in images_in_class[:MAX_IMAGES_PER_CLASS]:
            img_path = os.path.join(class_path, img_name)
            try:
                img = Image.open(img_path).resize(IMAGE_SIZE).convert('RGB')
                img_array = np.array(img) / 255.0 # Normalizziamo i pixel a [0, 1]
                data.append(img_array.flatten()) # Appiattiamo l'immagine in un vettore
                labels.append(class_name)
            except Exception as e:
                print(f"Errore durante il caricamento di {img_path}: {e}")
    if (i + 1) % 10 == 0:
        print(f"  Processed {i + 1}/{len(class_names)} classes...")

data = np.array(data)
labels = np.array(labels)

print(f"\nCaricamento completato in {time.time() - start_time:.2f} secondi.")
print(f"Caricate {len(data)} immagini totali.")
print(f"Dimensioni del dataset per t-SNE: {data.shape}")
print(f"Numero di classi caricate: {len(np.unique(labels))}")
print("-" * 50)

# 4.2. Scaling dei dati (utile per t-SNE)
print("Scalatura dei dati...")
scaler = StandardScaler()
scaled_data = scaler.fit_transform(data)
print("Scalatura completata.")
print("-" * 50)

# 4.3. Applicazione di t-SNE per la riduzione di dimensionalità
print("Applicazione di t-SNE (potrebbe richiedere del tempo)...")
# n_components=2 per 2D, random_state per riproducibilità
# perplexity e max_iter sono parametri importanti da impostare correttamente per ottenere una buona visualizzazione della nuvola di punti
# tsne = TSNE(n_components=2, random_state=42, perplexity=30, max_iter=1000, learning_rate=200)
# riportato max_iter a n_iter su Kagge notebook
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000, learning_rate=200)
tsne_results = tsne.fit_transform(scaled_data)
print("t-SNE completato.")
print("-" * 50)

# 4.4. Presentazione grafica della nuvola di punti
plt.figure(figsize=(16, 12))

# Mappiamo i nomi delle classi a valori numerici per la colorazione
unique_labels = np.unique(labels)
label_to_int = {label: i for i, label in enumerate(unique_labels)}
numeric_labels = np.array([label_to_int[label] for label in labels])

# sns.scatterplot è ottimo per questo
# Usiamo i punti ridotti da t-SNE, e il 'hue' per colorare per classe
sns.scatterplot(
    x=tsne_results[:, 0], y=tsne_results[:, 1],
    hue=labels, # Usa i nomi delle classi originali per la leggenda
    palette=sns.color_palette("tab10", len(unique_labels)), # Palette per il numero di classi
    legend="full",
    alpha=0.7,
    s=20 # Dimensione dei punti
)

plt.title('Nuvola di Punti delle Immagini di Frutta (Riduzione Dimensionalità con t-SNE)', fontsize=18)
plt.xlabel('Componente t-SNE 1', fontsize=14)
plt.ylabel('Componente t-SNE 2', fontsize=14)
plt.grid(True, linestyle='--', alpha=0.6)

# Posizioniamo la legenda fuori dal grafico se ci sono molte classi
if len(unique_labels) > 20:
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)
plt.tight_layout() # Aggiusta layout per evitare sovrapposizioni
plt.show()

# **Nuvola 3D**
Vista la visualizzazione della distribuzione in 2D, che mostra evidenti pattern ma che vediamo non bastare a spiegare la categorizzazione finale, aggiungiamo una dimensione in quanto evidentemente non stiamo considerando qualche feature importante nell'analisi della correlazione.
(Ci renderemo conto che anche in questo caso non abbiamo una visualizzazione ottimale ma nuovamente vediamo dei pattern riconoscibili)

In [ ]:
# 4.3.b Applicazione di t-SNE per la riduzione di dimensionalità
print("Applicazione di t-SNE (potrebbe richiedere del tempo)...")
# n_components=3 per 3D, random_state per riproducibilità
# perplexity e max_iter sono parametri importanti da impostare correttamente per ottenere una buona visualizzazione della nuvola di punti
# tsne = TSNE(n_components=3, random_state=42, perplexity=30, max_iter=1000, learning_rate=200)
# riportato max_iter a n_iter su Kagge notebook
tsne = TSNE(n_components=3, random_state=42, perplexity=30, n_iter=1000, learning_rate=200)
tsne_results = tsne.fit_transform(scaled_data)
print("t-SNE 3D completato.")
print("-" * 50)

# 4.4.b Presentazione grafica della nuvola di punti
plt.figure(figsize=(16, 12))
ax = plt.figure().add_subplot(projection='3d') # CREA UN SOTTO-GRAFICO 3D

# Ottieniamo le classi uniche e generiamo una palette di colori
unique_labels = np.unique(labels)
colors = sns.color_palette("tab10", len(unique_labels)) # Ottieniamo i colori da Seaborn
label_color_map = dict(zip(unique_labels, colors)) # Mappiamo le etichetta ai colori

# Visualizziamo ogni punto assegnando il colore in base alla sua classe
for i, label in enumerate(unique_labels):
    # Selezioniamo i punti che appartengono alla classe corrente
    indices = np.where(labels == label)
    ax.scatter(
        tsne_results[indices, 0],
        tsne_results[indices, 1],
        tsne_results[indices, 2],
        color=label_color_map[label], # Usiamo il colore mappato
        label=label, # Etichettiamo per la legenda
        alpha=0.7,
        s=20
    )

ax.set_title(f'Nuvola di Punti delle Immagini di Frutta (Top {NUM_CLASSES_TO_PLOT} Classi con t-SNE 3D)', fontsize=18)
plt.grid(True, linestyle='--', alpha=0.6)

# Posizioniamo la legenda fuori dal grafico
ax.legend(title='Classe di Frutta', bbox_to_anchor=(1.02, 1), loc='upper left', borderaxespad=0.)

plt.tight_layout()
plt.show()

# **Data Augmentation**
Sfruttando Keras con ImageDataGenerator procediamo a creare in tempo reale delle varianti delle immagini al fine di estendere il dataset fornito per l'addestramento della nostra IA.

Nel codice seguente useremo ImageDataGenerator per le seguenti trasformazioni:

* rescale=1./255: Normalizza i valori dei pixel da [0, 255] a [0, 1];

* rotation_range:Applica leggere rotazioni;

* width_shift_range, height_shift_range: Effettua degli spostamenti;

* shear_range: Applica delle deformazioni;

* zoom_range: Zoom in/out sulle immagini;

* horizontal_flip=True: Effettua il ribaltamento orizzontale delle immagini;

* fill_mode='nearest': Indica la modalità con cui riempire i pixel creati dalle trasformazioni.

Con le immagini generate usiamo flow_from_directory per caricare le immagini e assegnare le etichette. Specifichiamo il target_size (ad esempio (100, 100)), batch_size, class_mode='categorical'.

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# 5. Impostiamo i parametri per ImageDataGenerator e caricamento dati
TARGET_SIZE = (100, 100) # Dimensione a cui verranno ridimensionate tutte le immagini
BATCH_SIZE = 32          # Numero di immagini per batch

# 5.2. Procediamo con la creazione di ImageDataGenerator per il Training Set con Data Augmentation
# con le trasformazioni previste
print("Creazione del generatore di immagini per il training set...")
train_datagen = ImageDataGenerator(
    rescale=1./255,                 # Normalizza i pixel da [0, 255] a [0, 1]
    rotation_range=40,              # Ruota casualmente le immagini fino a 40 gradi
    width_shift_range=0.2,          # Sposta casualmente le immagini orizzontalmente (frazione della larghezza totale)
    height_shift_range=0.2,         # Sposta casualmente le immagini verticalmente (frazione dell'altezza totale)
    shear_range=0.2,                # Applica delle trasformazioni di "taglio" (shear transformation)
    zoom_range=0.2,                 # Applica zoom casuale all'interno delle immagini
    horizontal_flip=True,           # Ribalta casualmente le immagini orizzontalmente
    fill_mode='nearest'             # Strategia per riempire i pixel nuovi creati dalle trasformazioni (es. rotazioni)
                                    # 'nearest' riempie con il valore del pixel più vicino
)

# 5.3. Carichiamo le immagini generate dal Training Directory
print(f"Caricamento immagini dal percorso: {train_dir}")
train_generator = train_datagen.flow_from_directory(
    train_dir,                      # Percorso della directory di training
    target_size=TARGET_SIZE,        # Ridimensiona tutte le immagini a questa dimensione
    batch_size=BATCH_SIZE,          # Numero di immagini da restituire per ogni batch
    class_mode='categorical',       # Le etichette saranno in formato one-hot encoded (necessario per classificazione multi-classe)
    shuffle=True                    # Mescola l'ordine delle immagini per ogni epoca
)

print("\nGeneratore per il training set configurato con successo!")
print(f"Numero di classi trovate: {train_generator.num_classes}")
print(f"Nomi delle classi: {train_generator.class_indices}") # Mostra la mappatura tra nomi classi e indici numerici

# 5.4. Esempio: Visualizziamo alcuni batch di immagini per il Training generate
# Con questo blocco verifichiamo che le trasformazioni funzionino come previsto
# import matplotlib.pyplot as plt

print("\nVisualizzazione di alcune immagini trasformate (può richiedere qualche secondo)...")
plt.figure(figsize=(10, 10))
for images, labels in train_generator:
    for i in range(min(9, BATCH_SIZE)): # Visualizza fino a 9 immagini dal primo batch
        ax = plt.subplot(3, 3, i + 1)
        # ImageDataGenerator restituisce i pixel già in [0,1], quindi moltiplica per 255 se vuoi visualizzare
        # con imshow (che si aspetta valori in [0,1] o [0,255])
        plt.imshow(images[i])
        # Puoi provare a decodificare la label one-hot per visualizzare il nome della classe
        current_label_index = tf.argmax(labels[i]).numpy()
        current_label_name = list(train_generator.class_indices.keys())[list(train_generator.class_indices.values()).index(current_label_index)]
        plt.title(current_label_name)
        plt.axis("off")
    break # Interrompi dopo il primo batch
plt.tight_layout()
plt.show()

# 5.5 Creazione del generatore per il Test/Validation Set (SENZA data augmentation)
# È fondamentale NON applicare data augmentation al set di test/validazione.
# Vogliamo valutare il modello su immagini "pulite" e non alterate.
print("\nCreazione del generatore di immagini per il test/validation set (solo rescale)...")
test_datagen = ImageDataGenerator(rescale=1./255) # Solo normalizzazione

test_generator = test_datagen.flow_from_directory(
    test_dir,                       # Percorso della directory di test
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False                   # Non mescolare il test set per avere risultati riproducibili
)

print("\nGeneratore per il test set configurato con successo!")

# **Definizione del modello**
Terminate le operazioni sui dati ci dedichiamo al modello. Affrontando un problema di classificazione useremo una CNN che è particolarmente adatta alle nostre necessità essendo un'architettura di deep learning particolarmente adatta all'analisi di dati visuali, come le immagini.

Nel codice seguente definiremo l'architettura della nostra CNN utilizzando l'API Sequenziale di Keras

In [ ]:
# 6. Definizione del modello
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# 6.1 Impostazione dei parametri (Non vengono reimpostati quelli già impostati nei codici precedenti)
IMAGE_CHANNELS = 3       # 3 per i canali RGB delle immagini a colori
NUM_CLASSES = train_generator.num_classes

print(f"Dimensioni attese delle immagini in input: {TARGET_SIZE[0]}x{TARGET_SIZE[1]} con {IMAGE_CHANNELS} canali.")
print(f"Numero di classi per l'output: {NUM_CLASSES}")
print("-" * 50)

# 6.2. Architettura di Base della CNN
print("Definizione dell'architettura della CNN...")

model = Sequential([
    # Primo blocco Convoluzionale e Pooling
    Conv2D(32, (3, 3), activation='relu', input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], IMAGE_CHANNELS)),
    # Conv2D definisce lo strato della nostra rete ricevendo informazioni sul numero di feature da apprendere,
    # sulla dimensione della finestra convoluzionale che scansiona l'immagine, sulla funzione di attivazione
    # (usiamo la ReLU perché aiuta il modello ad apprendere relazioni non lineari ed è computazionalmente efficiente) e,
    # solo nel primo srato, le caratteristiche delle immagini da analizzare.
    MaxPooling2D((2, 2)),
    # Riduce la dimensione spaziale della feature map, aiuta a ridurre il numero di parametri, a rendere il modello
    # più robusto a piccole variazioni nella posizione degli oggetti e ad evitare l'overfitting.

    # Secondo blocco Convoluzionale e Pooling
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Terzo blocco Convoluzionale e Pooling
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Quarto blocco Convoluzionale e Pooling (opzionale, ma utile per feature più complesse)
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),

    # Strato di Dropout per prevenire l'overfitting
    Dropout(0.25),
    # Disattiva casualmente una percentuale (es. 25%) di neuroni in ogni passo di addestramento.
    # Questo simula l'addestramento di un insieme di reti sparse, forzando i neuroni ad apprendere percorsi
    # alternativi e a ridurre le co-dipendenze. Il risultato è un modello più robusto, che userò tutti i
    # neuroni disponibili durante la predizione, ma che è meno soggetto all'overfitting.

    # Appiattisce l'output degli strati convoluzionali per passarlo agli strati Dense
    Flatten(),
    # Converte l'outpud in 3 dimensioni (altezza, larghezza e filtri) in uno monodimensionale

    # Strati Fully Connected (Dense) di una Rete Neutrale a Percettroni Multistrato
    Dense(512, activation='relu'), # Uno strato nascosto
    Dropout(0.5),                  # Un altro strato di Dropout

    # Strato di output finale
    Dense(NUM_CLASSES, activation='softmax') # Il numero di neuroni deve essere uguale al numero di classi
                                             # 'softmax' per classificazione multi-classe:
                                             # converte gli output grezzi del neurone in probabilità che l'immagine
                                             # appartenga a ciascuna classe.
])

print("Architettura della CNN definita.")
print("-" * 50)

Una volta definita l'architettura del modello dobbiamo compilarlo assegnando una funzione di Loss ed una di ottimizzazione per poter addestrare il modello.
Per la tipologia di classificazione multi-classe in cui un oggetto appartiene o no ad una categoria, abbiamo selezionato l'ottimizzatore Adam e la funzione ci Loss CCE (Categorical Cross Entropy).

**Ottimizzatore**

L'ottimizzatore Adam si presta bene alla nostra esercitazione per l'efficienza computazionale e per la poca necessità di personalizzazione dei parametri, funzionando generalmente bene con le impostazioni di default.

**Funzione di Loss**

La funzione di Loss CCE da la misura di quanto una previsione sia errata, di quanto si discosti dalle Ground Truth (Le etichette fornite).
Cerchiamo di immaginare il funzionamento del nostro modello:

  Data un'immagine di mela, il modello produce delle probabilità per le tre classi:
  * Previsione del modello: [0.8, 0.1, 0.1] (80% mela, 10% banana, 10% arancia)
  * Etichetta reale (one-hot): [1.0, 0.0, 0.0] (è una mela)

La CCE calcola una penalità basata su quanto le probabilità previste per le classi sbagliate sono alte, e quanto la probabilità prevista per la classe corretta è bassa.

La formula è: L=−∑\[i=1 -> C\](yi*log(^yi))

C indica il numero delle classi, yi la ground truth (1 o 0), ^yi la nostra predizione.

*Il logaritmo fa si che quanto più la predizione si avvicina ad 1, tanto più il suo logaritmo si avvicina a 0 risultando meno penalizzante.*


In [ ]:
# 6.3. Compilazione del Modello
print("Compilazione del modello...")
model.compile(
    optimizer='adam',                   # Ottimizzatore Adam (Adaptive Moment Estimation), efficiente computazionalmente
                                        # e generalmente ben funzionante con parametri di default
    loss='categorical_crossentropy',    # Funzione di perdita per classificazione multi-classe one-hot encoded (Appartenenti o no alla classe)
    metrics=['accuracy']                # Metriche da monitorare durante l'addestramento (precisione)
)

print("Modello compilato con successo!")
print("-" * 50)

# 6.4. Riepilogo del Modello
print("Riepilogo dell'architettura del modello (model.summary()):\n")
model.summary()

print("\nDefinizione e compilazione del modello completate!")

# **Addestramento del Modello**
In questa fase, alimenteremo il modello con i dati di training e useremo il set di validazione per monitorare le sue prestazioni e prevenire l'overfitting.

Useremo anche i Callbacks per salvare il modello migliore e fermare l'addestramento se non ci sono più miglioramenti, evitando di andare in overfitting.

In [ ]:
# 7. Addestramento del modello
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 7.1 Parametri per l'addestramento
EPOCHS = 50 # Numero di epoche. Iniziamo con un numero ragionevole ed usiamo EarlyStopping.

# Percorso per salvare il modello migliore
checkpoint_filepath = '/content/drive/MyDrive/FruitDLProj/best_fruit_classifier_model.keras' # Salva in .keras format

print(f"Inizio addestramento del modello per {EPOCHS} epoche.")
print(f"Il modello migliore verrà salvato in: {checkpoint_filepath}")
print("-" * 50)

# 7.2. Callbacks

# 7.2.1. ModelCheckpoint: Salva il modello migliore
# Monitora la 'val_accuracy' (accuratezza sul set di validazione)
# mode='max' significa che cerchiamo di massimizzare questo valore
# save_best_only=True assicura che venga salvata solo la versione migliore del modello
# verbose=1 per mostrare un messaggio quando il modello viene salvato
checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

# 7.2.2. EarlyStopping: Ferma l'addestramento in anticipo se non ci sono miglioramenti
# monitor='val_accuracy' come per ModelCheckpoint
# patience=10 significa che aspetterà 10 epoche senza miglioramenti prima di fermarsi
# restore_best_weights=True ripristina i pesi del modello alla migliore epoca trovata
# verbose=1 per mostrare un messaggio quando l'addestramento si ferma
early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

# Lista di callbacks da passare a model.fit
callbacks_list = [checkpoint_callback, early_stopping_callback]

# 7.3. Addestramento Effettivo del Modello (model.fit())
history = model.fit(
    train_generator,                       # Il generatore di dati per il training
    epochs=EPOCHS,                         # Il numero massimo di epoche
    validation_data=test_generator,        # Il generatore di dati per la validazione (test set)
    callbacks=callbacks_list               # La lista dei callbacks definiti
    # steps_per_epoch e validation_steps possono essere omessi con flow_from_directory,
    # Keras li calcola automaticamente in base al numero di campioni e al batch_size.
)

print("\nAddestramento del modello completato!")
print("-" * 50)

# 7.4. Visualizzazione dell'andamento dell'addestramento
print("Visualizzazione dell'andamento di accuratezza e loss...")

# 7.4.1. Accedi alla cronologia dell'addestramento
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1) # Primo subplot per l'accuratezza
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')
plt.xlabel('Epoca')
plt.ylabel('Accuratezza')

plt.subplot(1, 2, 2) # Secondo subplot per la loss
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('Epoca')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()

print("\nAddestramento completato e cronologia visualizzata.")

# **Aggregazioni e semplificazioni per l'esercitazione**
Le risorse, anche acquistando le GPU A100, su Google Colab, con i dataset attuali richiederebbero oltre una settimana per l'addestramento, per cui, per riuscire a proporre il progetto ho valutato la necessità di ridurre la granularità di riconoscimento del modello, distinguere una mela da una banana ma non tra mela golden, mela cavendish, ecc.

Per ottenere questo risultato, prima di procedere con l'addestramento faremo aggregazioni ed un alleggerimento del dataset, limitando a 10 le classi desiderate, per poter raggiungere risultati osservabili in tempi appropriati alla portata del progetto, in vista del prossimo esame di luglio.

Dovremo modificare il codice che prepara i generatori di immagini (ImageDataGenerator.flow_from_directory) e che definisce il numero di classi nel modello (NUM_CLASSES), rispetto a quanto visto in precedenza.

Dopo una prima fase di test si sono rese necessarie ulteriori semplificazioni limitando fortemente il numero delle classi (scelte da me) e delle foto massime per classe.

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping

# 8. Funzione per aggregare e filtrare i nomi delle classi
# Questa funzione prende il nome di una sottocartella (es. 'Apple Braeburn')
# e restituisce il nome aggregato (es. 'Apple') filtrando solo le classi desiderate

def aggregate_and_filter_fruit_name(folder_name, desired_classes_set):
    # Prende la prima parola come nome generico del frutto
    aggregated_name = folder_name.split(' ')[0]
    if aggregated_name == 'Walnut': # Se il dataset ha "Walnut", lo mappiamo a "nut"
        aggregated_name = 'Nut'
    elif aggregated_name == 'Hazelnut': # Se il dataset ha "Hazelnut", lo mappiamo a "nut"
        aggregated_name = 'Nut'

    # Restituisce il nome aggregato solo se è nella lista delle classi desiderate
    if aggregated_name.lower() in desired_classes_set:
        return aggregated_name.lower() # Restituisce in minuscolo per coerenza
    else:
        return None # Indica che questa classe non deve essere inclusa

# 8.1. Definizione delle 10 classi desiderate
# Le classi saranno in minuscolo per facilitare il confronto
DESIRED_CLASSES = [
    "apple", "banana", "cherry", "nut", "orange", "strawberry"
]
DESIRED_CLASSES_SET = set(DESIRED_CLASSES) # Per lookup più efficienti
NUM_AGGREGATED_CLASSES = len(DESIRED_CLASSES)

print(f"Classi aggregate desiderate ({NUM_AGGREGATED_CLASSES}): {DESIRED_CLASSES}")
print("-" * 50)

# Limite immagini per classe
# Questo parametro fondamentale per ridurre il tempo di addestramento.
# Ogni classe avrà al massimo questo numero di immagini.
MAX_IMAGES_PER_CLASS_LIMIT = 150 # Puoi sperimentare con 200, 250, 300, 400

# 8.2. Preparazione dei dati per ImageDataGenerator con mappatura e filtraggio
def create_dataframe_for_generator_filtered(base_dir, class_aggregator_func, desired_classes_set, max_images_limit):
    filepaths = []
    labels = []
    for original_class_folder in os.listdir(base_dir):
        original_class_path = os.path.join(base_dir, original_class_folder)
        if os.path.isdir(original_class_path):
            aggregated_label = class_aggregator_func(original_class_folder, desired_classes_set)
            if aggregated_label is not None:
                # Ottieni tutti i nomi dei file immagine in questa sottocartella
                img_names = [img for img in os.listdir(original_class_path) if img.lower().endswith(('.jpg', '.jpeg', '.png'))]

                # Seleziona un sottoinsieme casuale di immagini se il conteggio supera il limite
                if len(img_names) > max_images_limit:
                    img_names = random.sample(img_names, max_images_limit) # Seleziona casualmente il numero massimo

                for img_name in img_names:
                    filepaths.append(os.path.join(original_class_path, img_name))
                    labels.append(aggregated_label)
    return pd.DataFrame({'filepaths': filepaths, 'labels': labels})

# Creazione dei DataFrames per train e test, ora filtrati e limitati
df_train = create_dataframe_for_generator_filtered(train_dir, aggregate_and_filter_fruit_name, DESIRED_CLASSES_SET, MAX_IMAGES_PER_CLASS_LIMIT)
df_test = create_dataframe_for_generator_filtered(test_dir, aggregate_and_filter_fruit_name, DESIRED_CLASSES_SET, MAX_IMAGES_PER_CLASS_LIMIT)


print(f"DataFrame di training creato con {len(df_train)} immagini (limitato a {MAX_IMAGES_PER_CLASS_LIMIT} per classe).")
print(f"DataFrame di test creato con {len(df_test)} immagini (limitato a {MAX_IMAGES_PER_CLASS_LIMIT} per classe).")
print("-" * 50)

# 8.3. ImageDataGenerator per il Training Set (con data augmentation) ---
TARGET_SIZE = (72, 72)
BATCH_SIZE = 16
EPOCHS = 6 # Riduciamo le epoche visto il problema semplificato

print("Creazione del generatore di immagini per il training set...")
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

train_generator = train_datagen.flow_from_dataframe(
    dataframe=df_train,
    x_col='filepaths',
    y_col='labels',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=DESIRED_CLASSES, # Usiamo la lista esplicita delle classi desiderate
    shuffle=True
)

print("\nGeneratore per il training set configurato con successo!")
print(f"Numero di classi aggregate trovate: {len(train_generator.classes)}")
print(f"Nomi delle classi aggregate: {train_generator.class_indices}")
print("-" * 50)

# 8.4. ImageDataGenerator per il Test/Validation Set (solo rescale)
print("Creazione del generatore di immagini per il test/validation set (solo rescale)...")
test_datagen = ImageDataGenerator(rescale=1./255)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=df_test,
    x_col='filepaths',
    y_col='labels',
    target_size=TARGET_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    classes=DESIRED_CLASSES, # Usiamo la lista esplicita delle classi desiderate
    shuffle=False
)

print("\nGeneratore per il test set configurato con successo!")
print("-" * 50)

# 8.5. Ridefinizione del Modello CNN
# Adattiamo NUM_CLASSES al nuovo numero di classi aggregate
IMAGE_CHANNELS = 3

print("Definizione dell'architettura della CNN (adattata alle classi aggregate)...")
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(TARGET_SIZE[0], TARGET_SIZE[1], IMAGE_CHANNELS)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(256, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    Flatten(),
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(NUM_AGGREGATED_CLASSES, activation='softmax') # USIAMO IL NUOVO NUMERO DI CLASSI AGGREGATE
])

print("Architettura della CNN definita.")
print("-" * 50)

# 8.6. Ricompilazione del Modello
print("Compilazione del modello...")
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)
print("Modello compilato con successo!")
print("-" * 50)

# 8.7. Riepilogo del nuovo Modello
print("Riepilogo dell'architettura del modello (model.summary()):\n")
model.summary()
print("\nDefinizione e compilazione del modello completate!")
print("-" * 50)


# 8.8. Callbacks per l'Addestramento
# Patch Google Colab
# checkpoint_filepath = '/content/drive/MyDrive/FruitDLProj/best_fruit_classifier_model.keras'
checkpoint_filepath = '/kaggle/working/best_fruit_classifier_model.keras'

checkpoint_callback = ModelCheckpoint(
    filepath=checkpoint_filepath,
    monitor='val_accuracy',
    mode='max',
    save_best_only=True,
    verbose=1
)

early_stopping_callback = EarlyStopping(
    monitor='val_accuracy',
    patience=3, # Abbassiamo per raggiungere prima possibile uno stato di operabilità per i test
    restore_best_weights=True,
    verbose=1
)

callbacks_list = [checkpoint_callback, early_stopping_callback]

# 8.9. Addestramento Effettivo del Modello
print("Inizio addestramento del modello aggregato...")
history = model.fit(
    train_generator,
    epochs=EPOCHS,
    validation_data=test_generator,
    callbacks=callbacks_list
)

print("\nAddestramento del modello aggregato completato!")
print("-" * 50)

# 8.10. Visualizzazione dell'andamento dell'addestramento
print("Visualizzazione dell'andamento di accuratezza e loss per il modello aggregato...")

acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs_range = range(len(acc))

plt.figure(figsize=(12, 6))

plt.subplot(1, 2, 1)
plt.plot(epochs_range, acc, label='Training Accuracy')
plt.plot(epochs_range, val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy (Aggregated Classes)')
plt.xlabel('Epoca')
plt.ylabel('Accuratezza')

plt.subplot(1, 2, 2)
plt.plot(epochs_range, loss, label='Training Loss')
plt.plot(epochs_range, val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss (Aggregated Classes)')
plt.xlabel('Epoca')
plt.ylabel('Loss')

plt.tight_layout()
plt.show()

print("\nAddestramento completato e cronologia visualizzata per le classi aggregate.")

Dal grafico si evince che alla sesta epoca il modello iniziava ad andare in overfitting, diminuendo il Loss sul set di training ma aumentando sul set di test per cui il modello che generalizza meglio e che è stato scelto è quello della quinta epoca.

# **Valutazione del Modello**


In [ ]:
# 9. Caricamento del Modello Migliore
checkpoint_filepath = '/kaggle/working/best_fruit_classifier_model.keras'

# Verifica se il file del modello esiste prima di caricarlo
if os.path.exists(checkpoint_filepath):
    print(f"Caricamento del modello migliore da: {checkpoint_filepath}")
    model = tf.keras.models.load_model(checkpoint_filepath)
    print("Modello caricato con successo!")
else:
    print(f"Attenzione: Il modello non è stato trovato in {checkpoint_filepath}.")
    print("Assicurati di aver addestrato il modello e che il checkpoint sia corretto.")
    print("Se hai appena finito l'addestramento, il 'model' dovrebbe essere già quello migliore.")
    # Se il modello non viene trovato e non è già in memoria, le prossime operazioni falliranno.
    # In un contesto reale, qui si dovrebbe gestire un errore o riaddestrare.

print("-" * 50)

# 9.1. Valutazione sul Test Set

if 'test_generator' in locals(): # Verifica se test_generator esiste già
    print("Valutazione del modello sul set di test...")
    # model.evaluate() restituisce la loss e le metriche (es. accuratezza)
    # Calcola automaticamente steps (total_samples / batch_size)
    loss, accuracy = model.evaluate(test_generator)

    print(f"\n--- Risultati della Valutazione sul Test Set ---")
    print(f"Test Loss: {loss:.4f}")
    print(f"Test Accuracy: {accuracy:.4f}")
    print("---------------------------------------------")

else:
    print("Errore: 'test_generator' non è stato trovato.")
    print("Assicurati di aver eseguito le celle di preparazione dei dati che creano 'test_generator'.")

# **Predizioni su nuove immagini**
Il modello prenderà le immagini inserite in uno specifico path, le preprocesserà ed elaborerà per assegnar loro una classe tra quelle per cui è addestrato.

In [ ]:
import tensorflow as tf
from PIL import Image
import numpy as np
import os
import matplotlib.pyplot as plt

# 10.1 Caricamento del Modello Addestrato
# Path del modello creato su Kaggle Notebook
model_path = '/kaggle/working/best_fruit_classifier_model.keras'

try:
    model = tf.keras.models.load_model(model_path)
    print(f"Modello caricato con successo da: {model_path}")
except Exception as e:
    print(f"Errore durante il caricamento del modello: {e}")
    print("Assicurati che il percorso del modello sia corretto e che il file esista.")
    # Esci o gestisci l'errore se il modello non può essere caricato
    exit()

print("-" * 50)

# 10.2. Parametri di Pre-processing (devono corrispondere a quelli del training) ---
TARGET_SIZE = (100, 100) # Dimensione a cui le immagini sono state ridimensionate durante l'addestramento


# 10.3 Definizione delle Classi (nell'ordine in cui il modello le ha apprese)
CLASS_NAMES = [
    "apple", "banana", "cherry", "nut", "orange", "strawberry"
]

print(f"Il modello classificherà tra le seguenti {len(CLASS_NAMES)} classi: {CLASS_NAMES}")
print("-" * 50)

# 10.4. Cartella delle Immagini da Classificare
# Crea una cartella in Google Drive e caricali le immagini che vuoi testare.
# Esempio: "/content/drive/MyDrive/test_images_for_prediction"
image_folder_path = '/content/drive/MyDrive/test_images_for_prediction' # <-- MODIFICA QUESTO PERCORSO

if not os.path.exists(image_folder_path):
    print(f"Errore: La cartella '{image_folder_path}' non esiste.")
    print("Crea la cartella e carica le immagini al suo interno.")
    exit()

# --- 5. Funzione per Pre-processare una singola immagine ---
def preprocess_image(image_path, target_size):
    img = Image.open(image_path).convert('RGB') # Assicurati che sia RGB
    img = img.resize(target_size) # Ridimensiona
    img_array = np.array(img) # Converti in array NumPy
    img_array = img_array / 255.0 # Normalizza i pixel a [0, 1]
    img_array = np.expand_dims(img_array, axis=0) # Aggiungi la dimensione del batch (1, H, W, C)
    return img_array

# --- 6. Esecuzione delle Previsioni ---
print(f"Inizio delle previsioni per le immagini in: {image_folder_path}")
plt.figure(figsize=(15, 15))
plot_count = 0

# Itera su tutti i file nella cartella specificata
for filename in os.listdir(image_folder_path):
    if filename.lower().endswith(('.png', '.jpg', '.jpeg')):
        image_path = os.path.join(image_folder_path, filename)
        
        # Pre-processa l'immagine
        processed_image = preprocess_image(image_path, TARGET_SIZE)
        
        # Esegui la previsione
        predictions = model.predict(processed_image)
        
        # Ottieni la classe prevista (indice della probabilità più alta)
        predicted_class_index = np.argmax(predictions[0])
        predicted_class_name = CLASS_NAMES[predicted_class_index]
        
        # Ottieni la probabilità associata alla classe prevista
        confidence = predictions[0][predicted_class_index]
        
        print(f"Immagine: {filename}")
        print(f"  Previsto: {predicted_class_name} (Confidenza: {confidence:.2f})")
        
        # --- Visualizzazione ---
        plot_count += 1
        ax = plt.subplot(4, 4, plot_count) # Adatta la griglia a quante immagini vuoi mostrare
                                          # Es. 4x4 per max 16 immagini
        original_img = Image.open(image_path) # Carica l'immagine originale per la visualizzazione
        plt.imshow(original_img)
        plt.title(f"{predicted_class_name}\n({confidence*100:.1f}%)", fontsize=10, color='green' if confidence > 0.7 else 'red')
        plt.axis('off')

        if plot_count >= 16: # Limita il numero di immagini visualizzate per evitare grafici enormi
            print("Visualizzazione limitata a 16 immagini. Elaborando le rimanenti senza mostrare il grafico.")
            break # Rimuovi questo break se vuoi visualizzare tutte le immagini

plt.tight_layout()
plt.show()

print("\nPrevisioni completate per le immagini nella cartella.")

# **CPU vs GPU**

Mi sono reso conto con lo stato del sistema che, a dispetto delle impostazioni, sta addestrando sulla CPU. Verifichiamo col prossimo blocco l'uso della GPU con alcune configurazioni.

In [ ]:
import tensorflow as tf

# Verifica se TensorFlow ha rilevato una GPU
gpus = tf.config.list_physical_devices('GPU')
if gpus:
    try:
        # Configura la crescita della memoria (sebbene non sia fondamentale è una buona pratica)
        tf.config.experimental.set_memory_growth(gpus[0], True)
        print(f"GPU rilevata: {gpus[0]}")
        print("Il tuo modello verrà addestrato sulla GPU.")
    except RuntimeError as e:
        # Errore di inizializzazione della GPU
        print(e)
        print("Errore durante l'inizializzazione della GPU. Il modello potrebbe usare la CPU.")
else:
    print("Nessuna GPU rilevata. Il tuo modello verrà addestrato sulla CPU.")

# Stampa la versione di TensorFlow (utile per debugging)
print(f"Versione di TensorFlow: {tf.__version__}")

# **Download del modello**

Script per scaricare il modello generato

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))


In [ ]:
download_file('/kaggle/working', 'out')
